In [2]:
import random
from matplotlib import pyplot as plt
import seaborn as sns

In [3]:
import numpy as np
import pandas as pd

In [7]:
import json
def load_intents():

    # intents file 
    intents = []
    with open("intents.json") as file:
        intents = json.load(file)

    df = pd.DataFrame({
        'Pattern' : [],
        'Tag' : []
    })

    i = 0
    for intent in intents['intents']:
        for pattern in intent['patterns']:
            
            sentence_tag = [pattern, intent['tag']]
            df.loc[i] = sentence_tag
            i += 1
    return df

df = load_intents()
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'intents.json'

In [ ]:
import os 
import json
def load_intents():
    '''
    Loads the intents.json file into a dataframe
    Returns:
        df: pd.DataFrame
    '''
    intents_file = 'intents.json'
    intents = None; 
    try:
        with open(intents_file) as file:
            intents = json.load(file)
    except Exception as e:
        raise Exception('No file named "intents.json" found.') from e
    
    tup_intent = []
    for intent in intents['intents']:
        for pattern in intent['patterns']:
            tup_intent.append((pattern, intent['tag']))
    
    return pd.DataFrame(tup_intent, columns =['Pattern', 'Tag'])
load_intents()

,Pattern,Tag
0,Hi,greeting
1,How are you?,greeting
2,Is anyone there?,greeting
3,Hello,greeting
4,Good day,greeting
...,...,...
102,what are the thing you can do,task
103,things you can do,task
104,what can u do for me,task
105,how u can help me,task


In [ ]:
labels = list(set(df['Tag']))

In [ ]:
# for training HuggingFace BERT model 
id2label = {}
label2id = {}
for id, label in list(enumerate(labels)):
    id2label[id] = label 
    label2id[label] = id 

In [ ]:
df['labels'] = df['Tag'].apply(lambda x: label2id[x.strip()])
df.head(1)

,Pattern,Tag,labels
0,Hi,greeting,0


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

X = df['Pattern'].tolist()
y = df['labels'].tolist()
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state = 123)

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizerFast
from transformers import TFDistilBertForSequenceClassification
from transformers import BertTokenizer, TFBertForSequenceClassification, BertConfig

C:\Users\mumle\AppData\Roaming\Python\Python312\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
C:\Users\mumle\AppData\Roaming\Python\Python312\site-packages\transformers\utils\generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", max_length=256)
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                      num_labels=len(labels), 
                                                      id2label=id2label, 
                                                      label2id = label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_encoding = tokenizer(X_train, truncation=True, padding=True)
test_encoding = tokenizer(X_test, truncation=True, padding=True)
full_data = tokenizer(X, truncation=True, padding=True)

In [ ]:
# pytorch dataloader 
import torch
from torch.utils.data import Dataset

class DataLoader(Dataset):
    
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_dataloader = DataLoader(train_encoding, y_train)
test_dataloader = DataLoader(test_encoding, y_test)
fullDataLoader = DataLoader(full_data, y_test)

HuggingFace transformer

In [ ]:
from transformers import DistilBertTokenizerFast
from transformers import TrainingArguments, Trainer

def compute_metrics(pred):

    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    
    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }

In [ ]:
training_args = TrainingArguments(
    output_dir='./output', 
    do_train=True,
    do_eval=True,
    num_train_epochs=100,              
    per_device_train_batch_size=32,  
    per_device_eval_batch_size=16,
    warmup_steps=100,                
    weight_decay=0.05,
    logging_strategy='steps',
    logging_dir='./multi-class-logs',            
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps", 
    load_best_model_at_end=True
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,                 
    train_dataset=train_dataloader,         
    eval_dataset=test_dataloader,            
    compute_metrics= compute_metrics
)

C:\Users\mumle\AppData\Roaming\Python\Python312\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,2.239600,1.985873,0.518519,0.365397,0.450000,0.416667
100,0.799100,1.123618,0.592593,0.371775,0.415476,0.400000
150,0.050500,1.176549,0.703704,0.538442,0.615476,0.575000
200,0.014600,1.250340,0.703704,0.538442,0.615476,0.575000
250,0.010500,1.286252,0.703704,0.538442,0.615476,0.575000
300,0.009100,1.295967,0.703704,0.538442,0.615476,0.575000


C:\Users\mumle\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\mumle\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\mumle\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(

TrainOutput(global_step=300, training_loss=0.5205593127012252, metrics={'train_runtime': 931.5241, 'train_samples_per_second': 8.588, 'train_steps_per_second': 0.322, 'total_flos': 139790298240000.0, 'train_loss': 0.5205593127012252, 'epoch': 100.0})

In [ ]:
q=[trainer.evaluate(eval_dataset=df) for df in [train_dataloader, test_dataloader]]
pd.DataFrame(q, index=["train","test"]).iloc[:,:5]

C:\Users\mumle\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\mumle\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


,eval_loss,eval_Accuracy,eval_F1,eval_Precision,eval_Recall
train,0.006582,1.000000,1.000000,1.000000,1.000
test,1.295967,0.703704,0.538442,0.615476,0.575


super overfit lol

In [ ]:
def predict(text):
    
    inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors="pt").to("cuda")
    outputs = model(**inputs)

    probs = outputs[0].softmax(1)
    pred_label_idx = probs.argmax()
    pred_label = model.config.id2label[pred_label_idx.item()]

    return probs, pred_label_idx, pred_label##

In [ ]:
text = "Hello"
predict(text)

(tensor([[9.9256e-01, 6.5877e-04, 8.7439e-04, 5.0778e-04, 7.0046e-04, 9.4617e-04,
          1.0292e-03, 4.6199e-04, 3.4292e-04, 3.3265e-04, 8.1774e-04, 7.6757e-04]],
        device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor(0, device='cuda:0'),
 'greeting')

In [ ]:
text = "I want a science fiction movie"
predict(text)

(tensor([[8.0490e-04, 9.9248e-01, 8.6499e-04, 8.0142e-04, 7.5018e-04, 5.5113e-04,
          3.2954e-04, 7.3960e-04, 1.2134e-03, 6.0609e-04, 3.7575e-04, 4.8655e-04]],
        device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor(1, device='cuda:0'),
 'genre')

In [ ]:
text = "I want to see a cool western"
predict(text)

(tensor([[5.9370e-04, 9.9205e-01, 7.8796e-04, 1.2133e-03, 7.6137e-04, 5.8391e-04,
          3.2854e-04, 6.4538e-04, 1.4118e-03, 7.5527e-04, 4.1436e-04, 4.5914e-04]],
        device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor(1, device='cuda:0'),
 'genre')

In [ ]:
text = "I want a romance"
predict(text)

(tensor([[6.9812e-04, 9.7493e-01, 9.8843e-04, 2.2197e-03, 1.9925e-03, 7.9983e-04,
          9.9588e-04, 1.1682e-03, 1.2140e-02, 1.9117e-03, 1.0083e-03, 1.1486e-03]],
        device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor(1, device='cuda:0'),
 'genre')

In [ ]:
text = "I want a romance movie where two people who hate each other fall in love"
predict(text)

(tensor([[1.2975e-03, 8.3524e-02, 1.5859e-03, 8.9995e-01, 2.0223e-03, 1.5410e-03,
          1.3635e-03, 2.1329e-03, 2.5842e-03, 1.9082e-03, 1.2881e-03, 8.0109e-04]],
        device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor(3, device='cuda:0'),
 'plot')

In [ ]:
text = "I Want to see a movie where two people who hate each other fall in love"
predict(text)

(tensor([[9.6722e-04, 2.9824e-02, 1.1189e-03, 9.5985e-01, 1.1840e-03, 9.7054e-04,
          8.1057e-04, 1.3870e-03, 1.1759e-03, 1.3494e-03, 8.4057e-04, 5.2626e-04]],
        device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor(3, device='cuda:0'),
 'plot')

In [ ]:
text = "idk can you give me something random"
predict(text)

(tensor([[0.0014, 0.0045, 0.0018, 0.0030, 0.0074, 0.0044, 0.0051, 0.0023, 0.9478,
          0.0031, 0.0173, 0.0018]], device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor(8, device='cuda:0'),
 'random')

In [ ]:
model_path = "chatbot"
trainer.save_model(model_path)
tokenizer.save_pretrained(model_path)

('chatbot\\tokenizer_config.json',
 'chatbot\\special_tokens_map.json',
 'chatbot\\vocab.txt',
 'chatbot\\added_tokens.json')

In [ ]:
from transformers import pipeline
model_path = "./chatbot"

model = BertForSequenceClassification.from_pretrained(model_path)
tokenizer= BertTokenizerFast.from_pretrained(model_path)
chatbot= pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)